In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2 
from sklearn import preprocessing
import os

#if linux
import matplotlib
matplotlib.use('TkAgg')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.version.cuda)
print("GPU:",torch.cuda.is_available())

11.7
GPU: True


In [2]:

name="pickle_linear.pkl"
file="/its/home/drs25/Documents/GitHub/RoboSkin/Code/Models/TacTip reader/dots/"

#load dataset
y=np.load(file+"augmented.npy")
X_train=np.load(file+"augmentedimages.npy")


print("Training dataset size",len(X_train))
print("Labels dataset size",len(y))
#shape correctly
SIZE=0.3
y=y.reshape((y.shape[0],y.shape[1]*2))
y=y/(255)*SIZE
h=X_train.shape[2]*SIZE
w=X_train.shape[1]*SIZE
#this is the part to apply preprocessing
X_train_grayscale = np.zeros((X_train.shape[0],int(h*w)))
for i in range(X_train.shape[0]):
    im=X_train[i].copy()
    im=cv2.resize(im,(int(h),int(w)),interpolation=cv2.INTER_AREA)
    im = np.float32(im)
    X_train_grayscale[i] = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY).flatten()/255


plt.imshow(cv2.resize(X_train[0],(int(h),int(w)),interpolation=cv2.INTER_AREA))
example=(y[0].reshape((y[-1].shape[0]//2,2))*255)
plt.scatter(example[:,0],example[:,1])
plt.show()

Training dataset size 1430
Labels dataset size 1430


In [3]:
X, data_test, Y, labels_test = train_test_split(X_train_grayscale, y, test_size=0.20, random_state=42)
X.shape,Y.shape

((1144, 27648), (1144, 266))

In [4]:

# Define the size of the input (n) and output (m) layers
n_inputs = X.shape[1]
m_outputs = Y.shape[1]

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(Y, dtype=torch.float32).to(device)

# Define the neural network model
class SimpleNeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size,layers=[1000,500,200],drop_out_prob=0.2):
        super(SimpleNeuralNetwork, self).__init__()
        self.fc=[nn.Linear(input_size, layers[0])]
        self.fc.append(nn.Sigmoid())
        self.fc.append(nn.Dropout(p=drop_out_prob))
        for i in range(1,len(layers)): #create layers 
                self.fc.append(nn.Linear(layers[i-1], layers[i]))
                self.fc.append(nn.Sigmoid())
                self.fc.append(nn.Dropout(p=drop_out_prob))
        self.fc.append(nn.Linear(layers[-1], output_size))
        self.fc_layers = nn.Sequential(*self.fc)
    def forward(self, x):
        x=self.fc_layers(x)
        return x

# Create the neural network
model = SimpleNeuralNetwork(n_inputs, m_outputs,drop_out_prob=0.1).to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss() #nn.MSELoss() #nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(), lr=0.01)

def train(model,num_epochs,output=True):
    loss_ar=[]
    for epoch in range(num_epochs):
        # Forward pass
        y_pred = model(X_tensor)

        # Compute the loss
        loss = criterion(y_pred, y_tensor)

        # Zero gradients, backward pass, and update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_ar.append(loss.item())
        #predict
        # Print the current loss to monitor training progress
        if epoch%1000==0 and output:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
    return np.array(loss_ar)
plt.title("Loss while training marker prediction model")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
loss=train(model,100000)
plt.plot(loss,label="Loss")
#plt.plot(b,label="Accuracy")
plt.legend(loc="upper right")
plt.show()

/its/home/drs25/miniconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Epoch [1/100000], Loss: 480.7980
Epoch [1001/100000], Loss: 473.4701
Epoch [2001/100000], Loss: 473.3919
Epoch [3001/100000], Loss: 473.3716
Epoch [4001/100000], Loss: 473.3607
Epoch [5001/100000], Loss: 473.3518
Epoch [6001/100000], Loss: 473.3474
Epoch [7001/100000], Loss: 473.3419
Epoch [8001/100000], Loss: 473.3369
Epoch [9001/100000], Loss: 473.3312
Epoch [10001/100000], Loss: 473.3150
Epoch [11001/100000], Loss: 473.2727
Epoch [12001/100000], Loss: 473.2505
Epoch [13001/100000], Loss: 473.2428


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "/its/home/drs25/Documents/GitHub/RoboSkin/Code/Models/TacTip reader/pytorchModelCross.pth")

In [ ]:
with torch.no_grad():
     model.eval()
     predictions = model(torch.tensor(data_test, dtype=torch.float32).to(device)).cpu().detach().numpy()
import random
N=random.randint(0,len(predictions)-1)
example=predictions[N]
example=(example.reshape((example.shape[0]//2,2))*255)/SIZE
image=data_test[N].reshape((int(w),int(h)))
image=cv2.resize(image,(X_train.shape[2],X_train.shape[1]),interpolation=cv2.INTER_AREA)
plt.imshow(image)
plt.scatter(example[:,0],example[:,1])
plt.show()